In [54]:
import pandas as pd
import os

In [55]:
filenames = [f for f in os.listdir() if 'intern' in f]
dfs = []
for f in filenames:
    df = pd.read_csv(f)
    dfs.append(df)

df = pd.concat(dfs, axis=1)
df = df.T
df.drop_duplicates(keep='first', inplace=True)
internships = df.T.columns.to_list()[1:]
internships = [int(i) for i in internships]

In [94]:
model = pd.read_csv('model_run-2022-01-13_02-03AM.csv')
tamu = pd.read_csv('tamu.csv')
indeed = pd.read_csv('indeed.csv')
usa_jobs = pd.read_csv('usa_jobs.csv')

print(indeed.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            228 non-null    int64 
 1   job_title             228 non-null    object
 2   company_name          228 non-null    object
 3   company_location      228 non-null    object
 4   job_link              228 non-null    object
 5   job_type              228 non-null    object
 6   salary                228 non-null    object
 7   full_job_description  228 non-null    object
dtypes: int64(1), object(7)
memory usage: 14.4+ KB
None


In [53]:
job_title = model.iloc[142].job_title
job_description = model.iloc[142].full_job_description
company_name = model.iloc[142].company_name
source = model.iloc[142].source
source_index = model.iloc[142].source_index
print(source_index)

if source == "tamu":
    link = tamu.iloc[142].URLs

if source == "usa_jos":
    link = usa_jobs.iloc[142].position_uri

if source == "indeed":
    link = indeed.iloc[142].Job_Link


157


In [88]:
def show_top_10(iloc_list, df = model):
    
    weighted_columns = [c for c in df.columns.to_list() if "W-" in c]
    data = df[weighted_columns]
    
    top_10_internships = []
    top_10_weights = []
    
    
    for i in iloc_list:
        top_10 = data.iloc[i].sort_values(ascending=False).head(11)
        ilocs = [int(s.replace('W-','')) for s in top_10.index.to_list()]
        
        top_10 = top_10[1:]
        ilocs = ilocs[1:]     

        top_10_weights.extend(top_10)
        top_10_internships.extend(ilocs)
        
    combined_df = pd.DataFrame(dict(
        ilocs = top_10_internships,
        weights = top_10_weights,
    ))

    # print(combined_df)
    combined_top = combined_df.sort_values(ascending=False, by='weights').head(10)
    

    return combined_top.ilocs.to_list()



In [89]:
top = show_top_10(internships)


In [95]:
def package_recommendations(top_internships):
    
    package = []

    for job in top_internships:
        job_title = model.iloc[job].job_title
        job_description = model.iloc[job].full_job_description
        company_name = model.iloc[job].company_name
        source = model.iloc[job].source
        source_index = model.iloc[job].source_index

        if source == "tamu":
            link = tamu.iloc[source_index].URLs

        if source == "usa_jos":
            link = usa_jobs.iloc[source_index].position_uri

        if source == "indeed":
            link = indeed.iloc[source_index].job_link

        recommendation = dict(
            job_title = job_title,
            job_description = job_description,
            company_name = company_name,
            link = link
        )

        package.append(recommendation)
    
    return package

In [96]:
package_recommendations(top)

[{'job_title': 'Biologicial Science Technician - U.S. Military Veterans Only',
  'job_description': "U.S. Military Veterans! Do you love working in the outdoors? Are you ready to start a career with the U.S. Fish & Wildlife Service? Keep reading to learn more about this great veteran-only (paid) internship opportunity.Position Summary: In partnership with the Mt. Adams Institute (MAI), the Warm Springs National Fish Hatchery will sponsor a VetsWork AmeriCorps Intern to serve as a Biological Science Technician and Maintenance Mechanic. The primary function of this facility is to collect adult salmon, spawn and rear them for release each year. The salmon are a vital resource for a tribal and recreational fishery as well as a contribution to the commercial ocean fishery. Our purpose is Tribal Trust Responsibility, to ensure that salmon return to the Warm Springs River as part of the treaty rights of the Confederated Tribes of the Warm Springs of Oregon, and for recreational and tribal fis